In [1]:
import numpy as np
import netCDF4 as nc

def read_nc(file,num):
    nf = nc.Dataset(file,'r')
    varname = nf.variables.keys()
    varname = list(varname)
    print(varname)
    lat = np.array(nf.variables[varname[1]][:])
    lon = np.array(nf.variables[varname[2]][:])
    var = np.array(nf.variables[varname[num]][:])
    var = np.where(var<-1000,np.nan,var)
    var = np.where(var>10**8,np.nan,var)
    return lat,lon,var

lat,lon,dataF = read_nc('/work/a06/yingying/Code/plot/exp_ils06/outflw/dataF.nc',3)


['time', 'lat', 'lon', 'Outflw']


In [7]:
print(np.nanmax(dataF))

20.98807


In [8]:
import numpy as np
import netCDF4 as nc
import os
import re
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap,BoundaryNorm
import matplotlib.cm as cm
import imageio

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.ticker as mticker
from cartopy.io import shapereader
from sklearn.metrics import mean_squared_error
import matplotlib.colors as mcolors

#%% change the settings
dahour = 6 
dahour_str = '{:02d}'.format(dahour) 
day1 = 1
day2 = 18
time_len = (day2-day1+1)*24
var_dis  = 'outflw' # check discharge
var_dep  = 'rivdph' # check water depth
ens_num  = 20

var_name = var_dis
#var_name = var_dep

if var_name == 'outflw':
    var_ylabel = 'Outflow (kg m-2 s-1)'
    var_dir = 'dis'
else:
    var_ylabel  = 'River Depth (m)'
    var_dir = 'wlv'

#exp  = 'exp_tej' # change runoff
#exp  = 'exp_ils' # change rainfall
#exp  = 'exp_wid' # change wid 
#exp  = 'exp_hgt' # change hgt 
exp  = 'exp_man' # change manning

df  = pd.read_excel("/work/a06/yingying/MLIT/MLIT_FlowGauge_all_info.xlsx",sheet_name='All_locationsV2')
site_name = np.array(df.iloc[:,0])
ref = np.array(df.iloc[:,1])

#%% read the location of observations
file_alloc = 'gauge_good_wlv.txt'
# read allocate information
id_info = list()
ix      = list()
iy      = list()
with open('/work/a06/yingying/obs/alloc/'+file_alloc,'r') as file:
    alloc_info = file.readlines()
    for i in range(1,len(alloc_info)):
        id_info = id_info+[int(alloc_info[i].split()[0])]
        ix      = ix+[int(alloc_info[i].split()[8])-1]
        iy      = iy+[int(alloc_info[i].split()[9])-1]
id_info  = np.array(id_info)
ilon     = np.array(ix)
ilat     = np.array(iy)

def cal_situ_ref(id_info_new):
    ref_new = np.full(np.size(id_info_new),np.nan)
    for i in range(0,np.size(id_info_new)):
        where_ref = np.where(site_name==id_info_new[i])
        ref_new[i]= ref[where_ref]
    ref_new = np.where((ref_new>998.99)&(ref_new<999.5),np.nan,ref_new)
    return ref_new
site_ref = cal_situ_ref(id_info)


def read_nc(file,num):
    nf = nc.Dataset(file,'r')
    varname = nf.variables.keys()
    varname = list(varname)
    print(varname)
    lat = np.array(nf.variables[varname[0]][:])
    lon = np.array(nf.variables[varname[1]][:])
    var = np.array(nf.variables[varname[num]][:])
    var = np.where(var<-1000,np.nan,var)
    var = np.where(var>10**8,np.nan,var)
    return lat,lon,var

def read_ens_nc(file):
    nf = nc.Dataset(file,'r')
    varname = nf.variables.keys()
    varname = list(varname)
    print(varname)
    lat = np.array(nf.variables[varname[0]][:])
    lon = np.array(nf.variables[varname[1]][:])
    var = np.array(nf.variables[varname[3]][:])
    var = np.where(var<-100,np.nan,var)
    return lat,lon,var

#%% read data
map_path = '/work/a06/yingying/CaMa_v411/cmf_v411_pkg/map/tej_01min/'
def read_bin(map_path,filename):
    with open(map_path+filename, 'rb') as f:
        day_array = np.fromfile(f, dtype=np.float32)
        day_array = day_array.reshape(1320,1500)
        day_array = np.where(day_array<-999,np.nan,day_array)
    return day_array

def read_nextxy(filename):
    with open(map_path+filename, 'rb') as f:
        day_array = np.fromfile(f, dtype=np.int32)
        day_array = day_array.reshape(2,1320,1500)
        day_array = np.where(day_array<-999,np.nan,day_array)
    return day_array

#%% read ens range
typeA = "A" # online DA
typeC = "C" # simulation
typeF = "F" # offline DA
output_path  = '/work/a06/yingying/Code/plot/'+exp+dahour_str+'/'

if var_name == 'outflw':
    lat,lon,disC_max = read_ens_nc(output_path+var_dis+'/data'+typeC+'_max01_20.nc')
    lat,lon,disC_min = read_ens_nc(output_path+var_dis+'/data'+typeC+'_min01_20.nc')
    lat,lon,disC_mean = read_ens_nc(output_path+var_dis+'/data'+typeC+'_mean01_20.nc')
    #lat,lon,disA_max = read_ens_nc(output_path+var_dis+'/data'+typeA+'_max01_20.nc')
    #lat,lon,disA_min = read_ens_nc(output_path+var_dis+'/data'+typeA+'_min01_20.nc')
    lat,lon,disA_mean = read_ens_nc(output_path+var_dis+'/data'+typeA+'_mean01_20.nc')
else:
    lat,lon,disC_max = read_ens_nc(output_path+var_dep+'/data'+typeC+'_max01_20.nc')
    lat,lon,disC_min = read_ens_nc(output_path+var_dep+'/data'+typeC+'_min01_20.nc')
    lat,lon,disC_mean = read_ens_nc(output_path+var_dep+'/data'+typeC+'_mean01_20.nc')
    #lat,lon,disA_max = read_ens_nc(output_path+var_dep+'/data'+typeA+'_max01_20.nc')
    #lat,lon,disA_min = read_ens_nc(output_path+var_dep+'/data'+typeA+'_min01_20.nc')
    lat,lon,disA_mean = read_ens_nc(output_path+var_dep+'/data'+typeA+'_mean01_20.nc')

basin=read_bin(map_path,'basin.bin')
up_area = read_bin(map_path,'uparea.bin')
elvmean = read_bin(map_path,'elv_mean.bin')
elvmean = np.where(elvmean<-100,np.nan,elvmean)
nextlon = read_nextxy('nextxy.bin')[0,:,:]-1
nextlat = read_nextxy('nextxy.bin')[1,:,:]-1
nextlon = np.where(nextlon<0,np.nan,nextlon)
nextlat = np.where(nextlat<0,np.nan,nextlat)

# read offline DA result
lat,lon,dataF = read_nc(output_path+var_dep+'/dataF.nc',3)    

#%% read rainfall data
def read_prep(file):
    nf = nc.Dataset(file,'r')
    varname = nf.variables.keys()
    varname = list(varname)
    print(varname)
    lat = np.array(nf.variables[varname[0]][:])
    lon = np.array(nf.variables[varname[1]][:])
    var = np.array(nf.variables[varname[3]][:])
    var = var*3600 # unit: mm/h 
    var = np.array(np.where(var<-900,np.nan,var))
    return lat,lon,var
lat,lon,rainf = read_prep('/work/a06/yingying/ils_data/tej_rain.nc')

def write_bin(var_each,varname,num):
    var_flat = var_each.flatten()
    var_bin  = var_flat.astype('int32').tobytes()
    # write .bin
    fname    = "basin"+varname+str(num)
    filebin  = map_path+fname+'.bin'
    with open(filebin,'wb') as file:
        file.write(var_bin)
    
obs_path = '/work/a06/yingying/MLIT/data/data_'+var_dir+'_hour2019/'
def read_obs(situ_id):
    filename = str(situ_id)+'.bin'
    if os.path.exists(filename):            
        with open(obs_path+filename, 'rb') as f:
            day_array = np.fromfile(f, dtype=np.float32)
            day_array = np.where(day_array<-900,np.nan,day_array)
        f.close()
        return day_array
    else:
        return -999


/tmp/ipykernel_447340/1525008990.py:70: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ref_new[i]= ref[where_ref]


['time', 'lat', 'lon', 'Outflw']
['time', 'lat', 'lon', 'Outflw']
['time', 'lat', 'lon', 'Outflw']
['time', 'lat', 'lon', 'Outflw']
['time', 'lat', 'lon', 'Outflw']
['time', 'lat', 'lon', 'Precip']


In [11]:
print(np.unique(disA_mean[:,165,1119]))

[-74.98143    -74.21477    -67.81659    -54.798576   -32.380817
 -31.513416   -28.164083   -26.43485    -23.663723   -20.212269
 -18.757557   -18.300068   -12.02899     -9.040554    -7.6120343
  -4.939837    -0.36004543   0.76947665   1.0936902    1.5502267
   1.5530876    1.7570908    1.8427396    1.8495991    1.8839633
   1.8856668    1.8910756    1.8945796    1.9122336    1.9202555
   1.9275447    1.9404919    1.960064     2.0343204    2.0425892
   2.079096     2.0962787    2.1117706    2.1227107    2.1511924
   2.183326     2.2983494    2.3726456    2.400998     2.4330058
   2.4659667    2.5159922    2.5690749    2.593171     2.60736
   2.6625595    2.6669087    2.6720643    2.6743445    2.701217
   2.7119212    2.7369266    2.7545342    2.789909     2.8005447
   2.8123581    2.8133411    2.8195686    2.823604     2.82665
   2.8300915    2.863336     2.8655806    2.8827052    2.8961601
   2.9082208    2.9208903    2.9454436    2.9520965    3.0058162
   3.0504146    3.0909896    3.1